In [2]:
import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from bs4 import BeautifulSoup

Команда 1: Покрышкин, Голохов

In [4]:
df_train = pd.read_csv('data/train.csv', encoding='utf8', sep='\t')
df_test = pd.read_csv('data/test.csv', encoding='utf8', sep='\t')

In [5]:
array_to_train=[]
for i in range(len(df_train)):
    array_to_train.append(BeautifulSoup(df_train.description[i], "lxml").text)

In [6]:
for i in range(len(df_test)):
    array_to_train.append(BeautifulSoup(df_test.description[i], "lxml").text)

In [7]:
def clearing(value):
    return BeautifulSoup(value, "lxml").text

df_train['descr_clear'] = df_train.description.map(clearing)
df_test['descr_clear'] = df_test.description.map(clearing)

In [8]:
from sklearn.cross_validation import train_test_split
y = df_train.target
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.1, random_state=67)

C:\Users\PC\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
X_train=X_train.descr_clear

In [10]:
X_test=X_test.descr_clear

In [11]:
model = TfidfVectorizer(sublinear_tf=True, use_idf=True, max_features=10000)

In [12]:
%%time
model.fit(array_to_train)

Wall time: 1min 54s


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
%%time
X_train_finish=model.transform(X_train)

Wall time: 57.1 s


In [14]:
%%time
X_test_finish=model.transform(X_test)

Wall time: 6.2 s


In [15]:
%%time
df_test_finish=model.transform(df_test.descr_clear)

Wall time: 55 s


In [16]:
print(X_train_finish.shape)
print(y_train.shape)
print(X_test_finish.shape)
print(y_test.shape)
print(df_test_finish.shape)

(180000, 10000)
(180000,)
(20000, 10000)
(20000,)
(170179, 10000)


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier

In [18]:
lr = LogisticRegression()
lr.fit(X_train_finish, y_train)
predict_lr_pr = lr.predict_proba(X_test_finish)

In [19]:
rfc = RandomForestClassifier(n_estimators=100, n_jobs=8)
rfc.fit(X_train_finish, y_train)
predict_rfc_pr = rfc.predict_proba(X_test_finish)

In [20]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train_finish, y_train)
predict_dtc_pr = dtc.predict_proba(X_test_finish)

KeyboardInterrupt: 

In [21]:
print('lr: ', roc_auc_score(y_test, predict_lr_pr[:,1]))
print('rfc: ', roc_auc_score(y_test, predict_rfc_pr[:,1]))
print('dtc: ', roc_auc_score(y_test, predict_dtc_pr[:,1]))

lr:  0.9890503750155781
rfc:  0.9917550883081274


NameError: name 'predict_dtc_pr' is not defined

In [22]:
predict_result_pr = rfc.predict_proba(df_test_finish)

In [23]:
pd.DataFrame(list(zip(df_test.id, predict_result_pr[:,1])), columns=['id','target']).to_csv('result.csv', index=False)